In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ada4b86f-1eb3-4559-b50b-740f87f012b7)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 4, Finished, Available, Finished)

In [3]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
.setTextCol("description")
.setKind ("SentimentAnalysis")
.setOutputCol("response")
.setErrorCol("error"))

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 5, Finished, Available, Finished)

In [4]:
#Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 6, Finished, Available, Finished)

In [5]:
display(result)

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a8d69115-8e05-4d17-8108-333b616c99e8)

In [6]:
#Create a sentiment column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 8, Finished, Available, Finished)

In [7]:
display(sentiment_df)

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e15f549f-e2dc-42fd-bd91-90a351a2a0d2)

In [8]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 10, Finished, Available, Finished)

In [9]:
display(sentiment_df_final)

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f6f10d1a-8c07-44d8-bbf3-32b1999b9ffb)

In [16]:
from pyspark.sql.functions import col, to_date
sentiment_df_final = df.withColumn("datePublished", to_date(col("datePublished"), 'dd-MMM-yyyy'))

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 18, Finished, Available, Finished)

In [10]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    # Write DataFrame as a Delta table
    sentiment_df_final.write.format("delta").saveAsTable(table_name)
    
except AnalysisException:

    print("Table Already Exists")

    # Create or replace a temporary view
    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    # Perform the MERGE operation
    spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING vw_sentiment_df_final AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title != target_table.title OR
            source_view.description != target_table.description OR
            source_view.category != target_table.category OR
            source_view.image != target_table.image OR
            source_view.provider != target_table.provider OR
            source_view.datePublished != target_table.datePublished
        )
        THEN UPDATE SET *

        WHEN NOT MATCHED THEN INSERT *
           
    """)

StatementMeta(, 4d9cb1d0-74a2-4092-a830-4b43c9d6c14e, 12, Finished, Available, Finished)

Table Already Exists
